# Text Pre Processor - Implantação

Este é um componente que utiliza a biblioteca [nltk](https://www.nltk.org/) e [ftfy](https://pypi.org/project/ftfy/) e [regex](https://docs.python.org/3/library/re.html) para pré processar textos que entrrão em outros componentes.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/nlp-text-pre-processor/pre_processor.py

In [ ]:
%%writefile Model.py
import joblib
import nltk
import pandas as pd
import numpy as np
from pre_processor import Preprocessor


class Model:
    
    def __init__(self):
        
        artifacts = joblib.load("/tmp/data/preprocessor.joblib")
        self.preprocessing_tasks = artifacts["preprocessing_tasks"]
        self.model_parameters = artifacts["model_parameters"]
        self.columns = artifacts["columns"]
        self.target = artifacts["target"]
        
            
    def class_names(self):
        return ['text']

    
    def predict(self, X, feature_names, meta=None):
        
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=self.columns)
        
        preprocessor = Preprocessor(self.preprocessing_tasks, self.model_parameters)
        texts = preprocessor.preprocess(df[self.target].to_numpy())
        
        return np.array(texts)

## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "review",
            "ftype": "categorical",
            "values": ["How are you doing?"]
        },
        {
            "name": "sentiment",
            "ftype": "categorical",
            "values": ["positive"]
        }
    ],
    "targets": []
}

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")